In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo
import pickle
from sklearn.preprocessing import RobustScaler,PowerTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'      

scalers = {}
# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values
    return pd.Series(original_values, index=group.index, name=group_name)


In [3]:

# Leer datos
#df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
#import dask.dataframe as dd
df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-PCA-Decompose.parquet') 
#df_final = df_final.compute()

df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)




In [4]:

## ESTO ES UN HACK PORQUE ESTA MAL ARMADA LA FUNCIÓN DE ESCALADO Y SOLO GUARDA LOS DATOS DE LA ÜLTIMA APLICACIÓN

with open(DATOS_DIR + '/scalers.pkl', 'rb') as file:
    scalers = pickle.load(file)

#df_test = df_final[(df_final['product_id'] == 21276) & (df_final['customer_id'] == 10029)][['product_id','tn', 'weight']]
#df_test['tn'] = df_test.groupby('product_id')['tn'].transform(inverse_scale_group)

#df_test

In [5]:

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-10-01']

In [6]:
df_true = df_true[['product_id','customer_id','tn']]
df_true

,product_id,customer_id,tn
periodo,,,
2019-12,20484,10010,0.000000
2019-12,20480,10008,0.476427
2019-12,20777,10382,0.333333
2019-12,20049,10187,0.000000
2019-12,20539,10003,1.650627
...,...,...,...
2019-12,20362,10258,0.000000
2019-12,20107,10089,0.923066
2019-12,20362,10262,0.000000


In [7]:
#df_true['tn'] = df_true.groupby(['product_id','customer_id'])['tn'].transform(inverse_scale_group)
#pred_true = df_true.groupby(['product_id'])['tn'].sum()
#pred_true.to_csv(DATOS_DIR+'/pred/TRUE-NOVEMBER.csv', index=True,header=True)


In [8]:
#df_final[df_final['product_id'] == 20402 & df_final['customer_id'] == 10479][['product_id','tn']]

In [9]:

#Filtro test
#df_final = df_final[df_final['product_id'] < 20013]
#df_final = df_final[df_final['product_id'] == 20072]

weight= df_final[['weight','product_id','customer_id']]
#df_final.drop(columns=['weight'], inplace=True)
#filtered_weight = weight[weight['weight'] != 0]
#weight_mean = filtered_weight.groupby('product_id')['weight'].mean().to_dict()
#weight['weight'] = weight['product_id'].map(weight_mean)

weight = weight.groupby(['product_id','customer_id'])['weight'].mean()
weight = weight.reset_index()
df_final = df_final.drop(columns=['weight'])

In [10]:
# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        'seed': 113,
        #'learning_rate': 0.2,
        #'bagging_fraction': 0.85,
        #'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0
for producto in products:
        # Filtrar los datos del producto
        df_producto = df_final[df_final['product_id'] == producto]
        for cluster in df_producto['cluster'].unique():
                print(f'Fitting and predicting for product_id: {producto} en el cluster {cluster}')
                # Filtrar los datos del producto
                df_cluster = df_producto[df_producto['cluster'] == cluster]
                
                product_weights = pd.merge(df_cluster.reset_index(), weight, on=['product_id', 'customer_id'], how='left')
                product_weights.set_index('periodo', inplace=True)
                weight_p = product_weights['weight']
                #############################################################################  
                #### Agrupar y escalar
                model, average_metric = labo.train_lightgbm_model_classic(df_cluster,params,metric='rmse',
                                                                weights=weight_p)
                print("Overall rmse metric: ", average_metric)
                # Predict values for the entire dataset using the trained models
                # Prepare last data points for prediction
                
                df_predict = df_cluster.loc['2018-01-01':'2019-10-01']
                df_predict_p = df_predict[df_predict['product_id'] == producto]
                last_data_points = df_predict_p[df_predict_p.index == df_predict_p.index.max()].copy()
                last_data_points.drop(columns=['tn_2'], inplace=True)
                # Predict the next month's value using the trained model
                predictions = labo.predict_next_month_customer(model, last_data_points)
                preds = predictions.groupby(['product_id','customer_id'])['tn_2'].transform(inverse_scale_group)
                predictions['tn'] = preds
                predictions.drop(columns=['tn_2'], inplace=True)
                predictions = predictions.reset_index()
                predictions =  predictions.groupby('product_id')['tn'].sum()
                predictions.columns = ['product_id', 'tn']
                predictions_all = pd.concat([predictions_all, predictions])
                print(predictions_all[-1:])


Fitting and predicting for product_id: 20263 en el cluster 3
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	validation's rmse: 5.38063
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[71]	validation's rmse: 3.19716
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[141]	validation's rmse: 1.72211
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[388]	validation's rmse: 4.66633
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[239]	validation's rmse: 1.08391
Overall rmse metric:  1.1748606247005033


C:\Users\Admin\AppData\Local\Temp\ipykernel_17276\2114168907.py:55: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_17276\2114168907.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])


            tn
20263  0.07259
Fitting and predicting for product_id: 20263 en el cluster 34
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[270]	validation's rmse: 3.31185
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[43]	validation's rmse: 3.3473
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[40]	validation's rmse: 3.68223
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[76]	validation's rmse: 4.46879
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[689]	validation's rmse: 8.63233
Overall rmse metric:  10.968358077594901
             tn
20263  0.850222
Fitting and predicting for product_id: 20263 en el cluster 12
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[192]	validation's rmse: 0.469168
Tr

ValueError: Cannot have number of folds=6 greater than the number of samples=5.

In [ ]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all =predictions_all.groupby('product_id')['tn'].sum()
predictions_all.to_csv(DATOS_DIR+'/pred/0031-prediccion-RSME_scaled_Robust-product_id-Outliers-Cluster.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)


Overall custom metric:  0.025267584246855608
